<a href="https://colab.research.google.com/github/Lemito66/Maestria/blob/master/Clasificaci%C3%B3n_clientes_Agency_Cargo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from datetime import date

In [ ]:
sales = pd.read_excel('Agency Cargo 2022Rev.xlsx')

# Preparación de los datos

In [ ]:
# Me aseguro que las fechas estén en formato de fecha.
sales['AWB Date'] = pd.to_datetime(sales['AWB Date'])
sales['ID CLIENTE'] = sales['ID CLIENTE'].str.strip()
sales['Total AWB'] = sales['Total AWB'].astype(float)
sales['Total AWB'] = sales['Total AWB'].round(2)

sales.sort_values(by='AWB Date', ascending = False).head()

,IT,Cod aerolínea,AWB No. SP,Wgt Charge,Tax AWB*,Due Carrier,Wgt Charge.1,Due Agent,AWB Date,Mes,...,BXS,PCS,Neto,Volumen,RATE,ID CLIENTE,Due Agent.1,AWB,Handling,Total AWB
6119,6120,235,235-6136 1005,8971.799805,0,35.000000,0.0,0.0,2022-12-31,12,...,52.00,113,2296,2361,3.80,21080321757996,12.5,9019.299805,0,9019.30
6117,6118,235,235-6136 0983,8611.610352,0,35.000000,0.0,0.0,2022-12-31,12,...,46.25,98,2783,2861,3.01,DMCC182926,59.0,8705.610352,0,8705.61
6749,6750,369,369-8604 1616,454.079987,0,113.000000,0.0,0.0,2022-12-31,12,...,4.00,8,159,176,2.58,91244343806,20.0,587.080017,0,587.08
6748,6749,369,369-8604 1605,831.299988,0,188.000000,0.0,0.0,2022-12-31,12,...,7.00,23,326,326,2.55,1725765406001,49.0,1068.300049,0,1068.30
2846,2847,145,145-0955 4031,2691.209961,0,224.059998,0.0,0.0,2022-12-31,12,...,14.25,28,762,823,3.27,1791405390001,99.0,3014.270020,0,3014.27


In [ ]:
sales[sales['AWB No. SP'].str.contains('157-8183 3323')]

,IT,Cod aerolínea,AWB No. SP,Wgt Charge,Tax AWB*,Due Carrier,Wgt Charge.1,Due Agent,AWB Date,Mes,...,BXS,PCS,Neto,Volumen,RATE,ID CLIENTE,Due Agent.1,AWB,Handling,Total AWB
4158,4159,157,157-8183 3323,4612.049805,0,27.5,0.0,0.0,2022-07-01,7,...,36.0,91,1665,1665,2.77,5003103483,19.0,4658.549805,0,4658.55


In [ ]:
sales[sales['ID CLIENTE'].str.contains('5003103483|')]

# Generación de las estadísticas

In [ ]:
# Calcular las estadisticas: Fecha de la última compra de cada cliente (Recency), número de meses con compra por cliente (Frequency), suma de las ventas por cada cliente (Monetary).
sales_stats = sales.pivot_table(values=['AWB Date', 'Mes', 'Total AWB'], index= ['ID CLIENTE'], aggfunc={'AWB Date': max, 'Mes': pd.Series.nunique, 'Total AWB': sum})

# Ordenar por la columna 'Total AWB' en orden descendente (mayor a menor)
sales_stats = sales_stats.sort_values(by='Total AWB', ascending=False)

print(sales_stats)

                 AWB Date  Mes   Total AWB
ID CLIENTE                                
5003103483     2022-12-30   12  7458349.81
21080321757996 2022-12-31   12  3091666.56
NL858608650B01 2022-12-31   12  2157416.39
01306202210202 2022-12-31    6  1627806.67
1712385        2022-12-29   12  1298623.89
...                   ...  ...         ...
110105311115   2022-04-10    2     2942.66
1791265009001  2022-11-13    2     2850.84
18697021716853 2022-05-22    1     1580.40
0590060240001  2022-11-21    1      571.10
C2043234600    2022-01-11    1      509.10

[100 rows x 3 columns]


Para una mejor visualización del dataframe y lo que significa cada columna, se actualiza los nombres de los campos:

1. Agregar una nueva columna llamada "R" que tenga el número de días que han pasado desde su última compra en "AWB Date"
2. Renombrar "Mes" a "F".
3. Renombrar "Total AWB" a "M".
4. Remover la columna "AWB Date" que ya no necesito.

In [ ]:
# Transformar la "fecha de la ultima compra" a "dias desde la ultima compra"
FREQUENCY = 'F'
RECENCY = 'R'
MONETARY = 'M'
sales_frm = sales_stats.copy()

today = sales['AWB Date'].max()
sales_frm[RECENCY] = (today - sales_frm['AWB Date']).dt.days

# Renombrar columnas para mejor lectura y remover las que ya no necesito
sales_frm = sales_frm.rename(columns={"Mes": FREQUENCY, "Total AWB": MONETARY})
sales_frm = sales_frm.drop(columns=['AWB Date'])

print(sales_frm)
print(type(sales_frm))

                 F           M    R
ID CLIENTE                         
5003103483      12  7458349.81    1
21080321757996  12  3091666.56    0
NL858608650B01  12  2157416.39    0
01306202210202   6  1627806.67    0
1712385         12  1298623.89    2
...             ..         ...  ...
110105311115     2     2942.66  265
1791265009001    2     2850.84   48
18697021716853   1     1580.40  223
0590060240001    1      571.10   40
C2043234600      1      509.10  354

[100 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>


# Calificación de clientes

Por cada columna F, R y M de los clientes se puede dar una calificación de la 'A' a la 'D' donde 'A' puede ser por ejemplo un cliente más fidelizado  y 'D' es un cliente menos fidelizado.


## Funciones de categorización
Estas son dos funciones que permiten generar una calificacón por cada columna F, R y M de un dataframe.

In [ ]:
# Función para asignar puntuaciones personalizadas a una columna
def add_score_for(df, column, q_values, reverse=False):
    min_limit = df[column].min()
    max_limit = df[column].max() + 1

    scale = ['A', 'B', 'C', 'D']
    if reverse:
        scale.reverse()

    df[f'{column}score'] = 'D'  # Inicializa con la peor puntuación 'D'

    q1, q2, q3 = q_values

    condition = (df[column] >= q3) & (df[column] <= max_limit)
    df.loc[condition, f'{column}score'] = scale[0]

    condition = (df[column] >= q2) & (df[column] < q3)
    df.loc[condition, f'{column}score'] = scale[1]

    condition = (df[column] >= q1) & (df[column] < q2)
    df.loc[condition, f'{column}score'] = scale[2]

    condition = (df[column] >= min_limit) & (df[column] < q1)
    df.loc[condition, f'{column}score'] = scale[3]

# Función para puntuar a los clientes
def score_customers(df, q_values):
    score_df = df.copy()
    for column, q_values_column, reverse in q_values:
        add_score_for(score_df, column, q_values_column, reverse)
    return score_df

# Tu DataFrame de ventas
sales_frm

# Definición de los valores de q para cada columna
q_values = [
    ('F', (df['F'].quantile(0.25), df['F'].quantile(0.50), df['F'].quantile(0.75)), False),
    ('M', (df['M'].quantile(0.25), df['M'].quantile(0.50), df['M'].quantile(0.75)), False),
    ('R', (30, 60, 90), True)
]

# Asignar las puntuaciones personalizadas
customers_scores_df = score_customers(sales_frm, q_values)
print(customers_scores_df)
print(q_values)

                 F           M    R Fscore Mscore Rscore
ID CLIENTE                                              
5003103483      12  7458349.81    1      A      A      A
21080321757996  12  3091666.56    0      A      A      A
NL858608650B01  12  2157416.39    0      A      A      A
01306202210202   6  1627806.67    0      C      A      A
1712385         12  1298623.89    2      A      A      A
...             ..         ...  ...    ...    ...    ...
110105311115     2     2942.66  265      D      D      D
1791265009001    2     2850.84   48      D      D      B
18697021716853   1     1580.40  223      D      D      D
0590060240001    1      571.10   40      D      D      B
C2043234600      1      509.10  354      D      D      D

[100 rows x 6 columns]
[('F', (3.0, 7.0, 10.0), False), ('M', (40983.797775268555, 111704.0947265625, 365367.0651855469), False), ('R', (30, 60, 90), True)]


# Interpretación de los datos
Esto es solo para probar qué información se puede sacar del dataframe con resultados, no tiene ningún objetivo en particular.

In [ ]:
def describe_customer_scores(customers_score):
  F = FREQUENCY + 'score'
  R = RECENCY + 'score'
  M = MONETARY + 'score'
  total = len(customers_score)

  clientes_AAA = customers_score[(customers_score[F] == 'A') & (customers_score[R] == 'A') & (customers_score[M] == 'A')]
  clientes_DDD = customers_score[(customers_score[F] == 'D') & (customers_score[R] == 'D') & (customers_score[M] == 'D')]

  print (f'{round(len(clientes_AAA) * 100 / total, 2)}% son clientes AAA.')
  print (f'{round(len(clientes_DDD) * 100 / total, 2)}% son clientes DDD.')


In [ ]:
describe_customer_scores(customers_scores_df)

13.0% son clientes AAA.
9.0% son clientes DDD.


In [ ]:
customers_scores_df['ScoreTOTAL'] = customers_scores_df['Fscore']+customers_scores_df['Mscore']+customers_scores_df['Rscore']
print(customers_scores_df)
print(type(customers_scores_df))

                 F           M    R Fscore Mscore Rscore ScoreTOTAL
ID CLIENTE                                                         
5003103483      12  7458349.81    1      A      A      A        AAA
21080321757996  12  3091666.56    0      A      A      A        AAA
NL858608650B01  12  2157416.39    0      A      A      A        AAA
01306202210202   6  1627806.67    0      C      A      A        CAA
1712385         12  1298623.89    2      A      A      A        AAA
...             ..         ...  ...    ...    ...    ...        ...
110105311115     2     2942.66  265      D      D      D        DDD
1791265009001    2     2850.84   48      D      D      B        DDB
18697021716853   1     1580.40  223      D      D      D        DDD
0590060240001    1      571.10   40      D      D      B        DDB
C2043234600      1      509.10  354      D      D      D        DDD

[100 rows x 7 columns]
<class 'pandas.core.frame.DataFrame'>


In [ ]:
#customers_scores_df['Cliente ID'] = customers_scores_df.index
#customers_scores_df.groupby('ScoreTOTAL')['Cliente ID'].nunique()
#customers_scores_df['Porcentaje'] = (customers_scores_df.groupby('ScoreTOTAL’)[‘ScoreTOTAL'].transform('count') / len(customers_scores_df)) * 100)

customers_scores_df['Cliente ID'] = customers_scores_df.index
unique_customers = customers_scores_df.groupby('ScoreTOTAL')['Cliente ID'].nunique()
#customers_scores_df['Porcentaje'] = (customers_scores_df.groupby('ScoreTOTAL')['ScoreTOTAL'].transform('count') / len(customers_scores_df)) * 100
customers_scores_df.groupby('ScoreTOTAL')['Cliente ID'].nunique()/customers_scores_df['Cliente ID'].nunique()*100

# Mostrar el resultado
print(unique_customers.sort_values(ascending = False))

ScoreTOTAL
AAA    13
DDD     9
ACA     7
CBD     6
CBA     6
ABA     6
CDD     6
BBA     6
BCA     5
CDA     4
CCA     4
CCD     3
DDB     3
DCA     2
BAD     2
AAC     2
CAD     2
BAA     2
DAD     1
DCD     1
DDA     1
AAD     1
CAA     1
BCD     1
BCC     1
BBD     1
BAC     1
ADA     1
ACC     1
BDA     1
Name: Cliente ID, dtype: int64


In [ ]:
#Clientes muy buenos y en riesgo de perderse. - Crucial de reactivar con ofertas especiales.
customers_scores_df[customers_scores_df['ScoreTOTAL'].str.contains('AAC|AAD')]

,F,M,R,Fscore,Mscore,Rscore,ScoreTOTAL,Cliente ID
ID CLIENTE,,,,,,,,
NL 825641706B01,10,1028021.85,63,A,A,C,AAC,NL 825641706B01
7726409702,10,719754.66,91,A,A,D,AAD,7726409702
1792878756,10,607074.26,61,A,A,C,AAC,1792878756


In [ ]:
#Clientes leales. - Programas de fidelización con valor agregado - Vacuum Cooling.
customers_scores_df[customers_scores_df['ScoreTOTAL'].str.contains('ABA|ACA')]

,F,M,R,Fscore,Mscore,Rscore,ScoreTOTAL,Cliente ID
ID CLIENTE,,,,,,,,
7282321474691,12,315828.06,0,A,B,A,ABA,7282321474691
1793192545001,11,217636.98,2,A,B,A,ABA,1793192545001
7725391636,12,198856.17,4,A,B,A,ABA,7725391636
1461861,10,146305.63,1,A,B,A,ABA,1461861
262511449890,10,135262.11,6,A,B,A,ABA,262511449890
1335977100012219,10,114841.74,1,A,B,A,ABA,1335977100012219
0317199881,10,111093.65,4,A,C,A,ACA,0317199881
824491484033000,12,91448.50,1,A,C,A,ACA,824491484033000
1014428GST501,12,80823.48,4,A,C,A,ACA,1014428GST501


In [ ]:
#Clientes nuevos. - Programas de fidelización para mantenerlos.
customers_scores_df[customers_scores_df['ScoreTOTAL'].str.contains('CBA|CAA')]

,F,M,R,Fscore,Mscore,Rscore,ScoreTOTAL,Cliente ID
ID CLIENTE,,,,,,,,
01306202210202,6,1627806.67,0,C,A,A,CAA,01306202210202
7734442069,3,335783.15,1,C,B,A,CBA,7734442069
9705107655,3,316518.16,1,C,B,A,CBA,9705107655
27319916-2-20,3,185599.41,3,C,B,A,CBA,27319916-2-20
1752018455001,5,166447.09,0,C,B,A,CBA,1752018455001
220940044277,3,166371.49,12,C,B,A,CBA,220940044277
NL863150263B01,3,132140.69,3,C,B,A,CBA,NL863150263B01


In [ ]:
#Clientes potencial. - Programas de fidelización con valor agregado - Vacuum Cooling.
customers_scores_df[customers_scores_df['ScoreTOTAL'].str.contains('DAD')]

,F,M,R,Fscore,Mscore,Rscore,ScoreTOTAL,Cliente ID
ID CLIENTE,,,,,,,,
6950172672,2,871415.66,316,D,A,D,DAD,6950172672


In [ ]:
#Prueba para visualizar las fechas del cliente con compras más recientes
sales[sales['ID CLIENTE'] == '6950172672'].sort_values(by='AWB Date', ascending=False).head()
#cliente_especifico = customers_scores_df.loc[customers_scores_df.index == '110105311115']
#print(cliente_especifico)

,IT,Cod aerolínea,AWB No. SP,Wgt Charge,Tax AWB*,Due Carrier,Wgt Charge.1,Due Agent,AWB Date,Mes,...,BXS,PCS,Neto,Volumen,RATE,ID CLIENTE,Due Agent.1,AWB,Handling,Total AWB
91,92,71,071-4413 0015,81604.250000,0,0.000000,0.0,0.0,2022-02-18,2,...,334.25,757,12046,13715,5.95,6950172672,59.0,81663.250000,0,81663.250000
92,93,71,071-4413 0026,51842.351562,0,0.000000,0.0,0.0,2022-02-18,2,...,179.00,358,8193,8713,5.95,6950172672,59.0,51901.351562,0,51901.351562
3425,3426,157,157-7529 5813,6289.149902,0,25.000000,0.0,0.0,2022-02-18,2,...,21.75,54,922,1057,5.95,6950172672,59.0,6373.149902,0,6373.149902
329,330,74,074-5153 7894,25930.099609,0,79.279999,0.0,0.0,2022-02-17,2,...,85.75,187,4142,4358,5.95,6950172672,59.0,26068.380859,0,26068.380859
3424,3425,157,157-7529 5802,6744.279785,0,25.000000,0.0,0.0,2022-02-17,2,...,64.75,163,2190,2426,2.78,6950172672,59.0,6828.279785,0,6828.279785
